In [ ]:
from itertools import starmap
from functools import partial
from collections import namedtuple
import re
import os
import datetime
import pathlib

from IPython import display
import traitlets
from ipywidgets import Button, Text, SelectMultiple, HBox, VBox, Textarea, Image, Label, HTML, Dropdown, Checkbox
import pandas
import jinja2
from pony import orm

from progressbar import ProgressWithTitle

import db

In [ ]:
os.chdir('data')
db.use_db(str(pathlib.Path.cwd() / 'course.sqlite'))

In [ ]:
### Javascript kludges to add autocompletion to text boxes and enable multiple selections

autocomplete = jinja2.Template("""
  $( function() {
    var availableTags = [
    {%- for option in options %}
        "{{ option }}",
    {%- endfor %}
    ];
    $( ":input[{{ selector }}]" ).autocomplete({
      minLength: 3,
      source: availableTags
    });
  } );
""")

In [ ]:
%%HTML
<style>
.widget-hbox .widget-label {
    max-width: unset ;
}
</style>

# Widget definitions

In [ ]:
### Views -- all the individual widgets and UI state

class AppState(traitlets.HasTraits):
    # Immutable
    students = traitlets.List(traitlets.Unicode())
    exams = traitlets.List(read_only=True)

    exam_id = traitlets.Integer()

    submission_id = traitlets.Integer()
    student = traitlets.Unicode()
    email = traitlets.Unicode()
    
    validated = traitlets.Bool()
    
    image = traitlets.Bytes()

    # --- Defaults and validation ---
    ## students
    @traitlets.default('students')
    def _default_students(self):
        with orm.db_session:
            return tuple(starmap('{}, {}, {}'.format,
                                 orm.select((s.id, s.first_name, s.last_name)
                                            for s in db.Student)))
    
    ## exams
    @traitlets.default('exams')
    def _default_exams(self):
        with orm.db_session:
            exams = orm.select(e for e in db.Exam).order_by(lambda e: e.id)
            return [(e.name, e.id) for e in exams]

    ## exam_id
    @traitlets.default('exam_id')
    def _default_exam_id(self):
        with orm.db_session:
            # Nonempty exam with the most recent yaml
            return sorted(db.Exam.select(lambda e: len(e.submissions)),
                          key=lambda e: os.path.getmtime(e.yaml_path))[-1].id

    @traitlets.validate('exam_id')
    def _valid_exam(self, proposal):
        with orm.db_session():
            if db.Exam.get(id=proposal['value']) is None:
                raise traitlets.TraitError('Unknown exam id.')
        return proposal['value']

    ## submission_id
    @traitlets.default('submission_id')
    def _default_submission_id(self):
        with orm.db_session:
            sid = (orm.select(s.id for s in db.Submission
                             if s.exam.id == self.exam_id 
                                and not s.signature_validated)
                      .first())
            return sid or -1

    @traitlets.validate('submission_id')
    def _valid_submission_id(self, proposal):
        sid = proposal['value']
        if sid == -1:
            return sid
        with orm.db_session():
            sub = db.Submission.get(id=sid)
            if sub is None:
                raise traitlets.TraitError('Unknown submission id.')
            if sub.exam.id != self.exam_id:
                raise traitlets.TraitError('Submission from a different exam.')
        return sid
    
    @traitlets.default('validated')
    def _default_validated(self):
        with orm.db_session():
            return db.Submission[self.submission_id].signature_validated
    
    @traitlets.default('student')
    def _default_student(self, sid=None):
        sid = sid or self.submission_id
        with orm.db_session:
            stud = db.Submission[sid].student
            if stud is None:
                return ''

            return '{}, {}, {}'.format(stud.id, stud.first_name, stud.last_name)
    
    @traitlets.default('email')
    def _default_email(self):
        match = re.match(r'(\d{7}), (.+), (.+)', self.student)
        if match is None:
            return ''
        else:
            number, *_ = match.groups()

        with orm.db_session:
            stud = db.Student.get(id=int(number))
            if stud is not None:
                return stud.email or ''
            return ''

    @traitlets.default('image')
    def signature_image(self, sid=None):
        if sid is None:
            sid = self.submission_id
        with orm.db_session:
            sub = db.Submission[sid]
            try:
                if sub.signature_image_path != 'None':
                    with open(sub.signature_image_path, 'rb') as f:
                        return f.read()
                else:
                    *_, widgets = db.read_yaml(sub.exam.yaml_path)
                    first_page = next(p.path for p in sub.pages if 'page1.' in p.path)
                    image = db.get_widget_image(first_page, widgets.loc['studentnr'])
            except Exception:
                return b''
                
        return image

    ## --- Trait relations ---
    @traitlets.observe('exam_id')
    def _change_exam(self, change):
        self.submission_id = self._default_submission_id()
        
    @traitlets.observe('submission_id')
    def _change_submission(self, change):
        self.image = self.signature_image()
        self.student = self._default_student()
        self.email = self._default_email()
        self.validated = self._default_validated()

    def commit_student(self):
        if is_not_student(self.student):
            return
        sid = self.submission_id
        # Parse the student entry
        student_nr, first_name, last_name = re.match(r'(\d{7}), (.+), (.+)',
                                                     self.student).groups()
        first_name = first_name.strip()
        last_name = last_name.strip()
        email = self.email.strip()
        with orm.db_session:
            stud = db.Student.get(id=int(student_nr)) or\
                   db.Student(id=student_nr, first_name=first_name, last_name=last_name)
            if first_name:
                stud.first_name = first_name
            if last_name:
                stud.last_name = last_name
            if email:
                stud.email = email

            db.Submission[sid].student = stud
            db.Submission[sid].signature_validated = True
        
    @traitlets.observe('student')
    def _update_email(self, change):
        self.email = self._default_email()

    # --- Navigation
    def next_submission(self, unchecked_only=False):
        with orm.db_session:
            own_sub = db.Submission[self.submission_id]
            subs = (db.Submission
                      .select()
                      .filter(lambda s: s.exam.id == self.exam_id 
                                        and s.id > self.submission_id
                                        and (not unchecked_only or not s.signature_validated))
                      .order_by(lambda s: s)
                   )
            result = subs.first()
            if result is None:
                # We're at the last (unchecked) submission.
                subs = (db.Submission
                          .select()
                          .filter(lambda s: s.exam.id == self.exam_id
                                            and (not unchecked_only or not s.signature_validated))
                          .order_by(lambda s: s)
                       )
                result = subs.first()
            if result is not None:
                self.submission_id = result.id
    
    def submissions_number(self, exam_id=None):
        with orm.db_session:
            return orm.count(db.Submission.select(lambda s: s.exam.id == (exam_id and self.exam_id)))

    def progress(self, sid=None):
        with orm.db_session:
            return orm.count(db.Submission.select(lambda s: s.exam.id == self.exam_id
                                                            and s.signature_validated))


state = AppState()

def is_not_student(student):
    try:
        student_nr, first_name, last_name = re.match(r'(\d{7}), (.+), (.+)',
                                                     student).groups()
        return False
    except AttributeError:
        return True


exam = Dropdown(options=state.exams)
traitlets.link((state, 'exam_id'), (exam, 'value'))

validate_and_next = Button(icon='fa-check-circle', description=' validate & next')
@validate_and_next.on_click
def _validate_and_next(sender=None):
    state.commit_student()
    state.next_submission(unchecked_only=True)

next_sub = Button(icon='fa-step-forward', description=' next')
next_sub.on_click(lambda change: state.next_submission())

validated = Checkbox(description='Checked', disabled=True)
traitlets.link((state, 'validated'), (validated, 'value'))

student_info = Text(placeholder='number, first, last')
traitlets.link((state, 'student'), (student_info, 'value'))

email_info = Text(placeholder='email')
traitlets.link((state, 'email'), (email_info, 'value'))

signature = Image(format='png', width='100%')
traitlets.directional_link((state, 'image'), (signature, 'value'))

number_progress = ProgressWithTitle(
    min=0,
    step=1,
    title='Progress:',
    bar_style='success',
    orientation='horizontal',
)

traitlets.directional_link((state, 'exam_id'), (number_progress, 'max'), state.submissions_number)
traitlets.directional_link((state, 'submission_id'), (number_progress, 'value'), state.progress)
traitlets.directional_link((state, 'submission_id'), (number_progress, 'value'), state.progress);

# Layout

In [ ]:
VBox([HBox([exam, number_progress]),
      HBox([validated, student_info, email_info, validate_and_next, next_sub])])

In [ ]:
signature

In [ ]:
# apply javascript kludges
# must trigger after the widgets were rendered

selector = "placeholder='{}'".format(student_info.placeholder)
display.display_javascript(display.Javascript(autocomplete.render(selector=selector, options=state.students)))